# Mendelian Randomization Analyses

This code was run in the UK Biobank Research Analysis Platform to create the MR models (IVW and median estimator). This analysis was repeated for each educational attainment group separately. For simplicity, we only show the model fitting on the full dataset here, as the actual process involved repeating this four times with the groups created in the Observational Analyses code.

In [ ]:
dx download UPDATEDFULLPROCDATASET.csv


In [ ]:
install.packages("lubridate")
library(lubridate)

install.packages('data.table')
library(data.table)


install.packages('timereg')
library(timereg)

install.packages('MendelianRandomization')
library(MendelianRandomization)


In [ ]:


dataset <- read.csv("UPDATEDFULLPROCDATASET.csv")

colnames(dataset)
ncol(dataset)


dataset[ , c(264:360)]
# Isolates the 97 variants

dataset$CensorDate <- fifelse(dataset$REGION == "Scotland", as.Date("2021-07-31"),
                             fifelse(dataset$REGION == "Wales", as.Date("2018-02-28"), as.Date("2021-09-30")))

                             
summary(as.Date(dataset$CensorDate))


In [ ]:

# Creating for loop to create clones of variants from 1 to 96 as V1 to V96 - v fast so no issue
for(i in 1:97) {
  nam <- paste("V", i, sep = "")
  
  dataset$nam <- dataset[ , (i + 263)]
  
  # Renaming column
  names(dataset)[names(dataset) == "nam"] <- nam
  
  
}


In [ ]:

# Creating V1 to V97 variants for now... Easier to reference
summary(dataset$V1)
summary(dataset$V97)
# Look to be 0 to 2


In [ ]:
# Creating empty vectors to store coefficients and standard errors
Bx <- matrix(NA, nrow = 97, ncol = 1)
Bxse <- matrix(NA, nrow = 97, ncol = 1)



In [ ]:
# Number of genetic variants
n <- 97


# run n regressions
Stage1Regs <- lapply(1:n, function(x) glm(FirstBMI ~ dataset[,x+361] + AgeBaseline + EverSmoke + AssayType + Biological.Sex +
                                            PC1 + PC2 + PC3 + PC4 + PC5 + PC6 + PC7 + PC8 + PC9 + PC10, data = dataset))


names(summary(Stage1Regs[[1]]))
# Figure out where coefficients and SEs lie now

summary(Stage1Regs[[1]])$coefficients[2,1]
summary(Stage1Regs[[1]])$coefficients[2,2]
summary(Stage1Regs[[1]])$coefficients



In [ ]:
# Extract coefficients and SEs
summaries <- lapply(Stage1Regs, summary)
Bx <- lapply(summaries, function(x) x$coefficients[2,1])
Bxse <- lapply(summaries, function(x) x$coefficients[2,2])

head(Bx)
head(Bxse)
# Confirmed this works

summary(as.numeric(Bx))
summary(as.numeric(Bxse))
# List makes summary behave weird, so force to numeric vector to view range of coeffs/ses

is.list(Bx)
is.list(Bxse)

# Unlist and turn into numeric vectors
Bx <- as.numeric(unlist(Bx))
Bxse <- as.numeric(unlist(Bxse))


typeof(Bx)
typeof(Bxse)


In [ ]:

# Creating empty vectors
By <- matrix(NA, nrow = 97, ncol = 1)
Byse <- matrix(NA, nrow = 97, ncol = 1)


In [ ]:

n <- 97

# run n regressions
Stage2Regs <- lapply(1:n, function(x) aalen(Surv(TimeYear, StatBinary) ~ const(dataset[,x+361]) + const(AgeBaseline) + const(EverSmoke) + const(AssayType) + const(Biological.Sex) +
                                              const(PC1) + const(PC2) + const(PC3) + const(PC4) + const(PC5) + const(PC6) + const(PC7) + const(PC8) + const(PC9) + const(PC10), robust = 0, data = dataset))



head(Stage2Regs)




In [ ]:
# Extract coefficients and SEs
coefs <- lapply(Stage2Regs, coef)
By <- lapply(coefs, function(x) x[1,1])
Byse <- lapply(coefs, function(x) x[1,2])

head(By)
head(Byse)

is.list(Byse)

# Unlist and turn into numeric vectors
By <- as.numeric(unlist(By))
Byse <- as.numeric(unlist(Byse))

typeof(Byse)


In [ ]:
# Creating mr_input object
mrinput <- mr_input(
  bx = Betas$Bx,
  bxse = Betas$Bxse,
  by = Betas$By,
  byse = Betas$Byse,
  exposure = "Yin",
  outcome = "Yang"
)

In [ ]:
# FIRST running unweighted/unpenalized IVW Model
ivw1 <- mr_ivw(mrinput,
               model = "default",
               penalized = FALSE,
               robust = FALSE,
               weights = "simple",
               alpha = 0.05)




ivw1





coeff <- ivw1@Estimate
lower <- ivw1@CILower
higher <- ivw1@CIUpper


In [ ]:

# ---------
# Median Estimator Results
# ---------

# FIRST weighted median
med1 <- mr_median(mrinput,
  weighting = "weighted",
  distribution = "normal",
  alpha = 0.05,
  iterations = 10000) # Set iterations for calculating SEs via bootstrap




coeffmed <- med1@Estimate
lowermed <- med1@CILower
highermed <- med1@CIUpper

